In [466]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import math
from numpy import std

In [454]:
conn = sqlite3.connect('DB.db')
c = conn.cursor()

# Task 1

## Part 1

In [455]:
def get_all(table):
    c.execute(f"SELECT * FROM {table}")
    return c.fetchall()

In [456]:
get_all('prices')[0:10]

[('2018-11-09', 'W', 92.93),
 ('2013-09-30', 'J', 14.63),
 ('2015-12-27', 'E', 304.53),
 ('2019-03-22', 'D', 210.91),
 ('2015-05-13', 'O', 23.89),
 ('2012-02-27', 'W', 135.74),
 ('2016-07-10', 'D', 88.07),
 ('2019-01-17', 'M', 758.44),
 ('2016-07-16', 'K', 1136.31),
 ('2017-03-23', 'E', 39.86)]

In [457]:
c.execute("SELECT DISTINCT product from prices ORDER BY product")
product_names = c.fetchall()
latest = []
for i in range(len(product_names)):
    c.execute(f"SELECT * FROM prices WHERE product='{product_names[i][0]}' ORDER BY date_start")
    rows = c.fetchall()
    latest.append(rows[-1])
latest[0:5]

[('2017-04-01', 'A', 111.23),
 ('2019-06-10', 'B', 310.31),
 ('2019-06-25', 'C', 121.17),
 ('2019-04-19', 'D', 64.53),
 ('2019-01-15', 'E', 293.38)]

In [473]:
sum=0
for i in range(len(latest)):
    sum += latest[i][2]
avg = sum/len(latest)
print('The average of the newest prices is %.3f' % avg)

The average of the newest prices is 366.905


In [138]:
with open ('res1.txt', 'w') as w:
    w.write('%.3f' % avg)

## Part 2

In [459]:
def my_func(product: str):
    c.execute(f"SELECT * FROM prices WHERE product='{product}' ORDER BY date_start")
    alist = c.fetchall()
    dates, prices= [], []
    for i in range(len(alist)):
        dates.append(alist[i][0])
        prices.append(alist[i][2])
    return dates, prices

In [460]:
print(my_func('Z'))

(['1970-01-01', '2010-04-12', '2012-06-28', '2013-01-10', '2013-06-03', '2015-04-17', '2015-07-11', '2015-07-30', '2015-12-07', '2016-02-07', '2016-04-21', '2016-07-12', '2017-11-09', '2018-12-08', '2019-07-02', '2019-08-23'], [119.21, 129.89, 76.37, 103.83, 465.95, 352.25, 428.15, 172.66, 33.51, 853.49, 7.43, 86.85, 553.63, 348.29, 167.78, 13.62])


# Task 2

In [146]:
print('Total number of records in sales table:', len(get_all('sales')))
get_all('sales')[0:10]

Total number of records in sales table: 91624


[('A', '2018-01-14', 9),
 ('H', '2013-11-08', 20),
 ('H', '2013-11-06', 1),
 ('P', '2014-11-06', 20),
 ('H', '2016-05-30', 26),
 ('R', '2015-10-29', 2),
 ('F', '2017-08-25', 6),
 ('J', '2015-04-14', 12),
 ('A', '2015-05-14', 5),
 ('L', '2011-08-26', 0)]

In [405]:
c.execute("""CREATE TABLE revenue (
            product TEXT,
            date TEXT,
            amount INTEGER,
            revenue REAL
            )""")
conn.commit()

In [ ]:
def revenue_for_product(product):
    
    c.execute(f"SELECT * FROM prices WHERE product='{product}' ORDER BY date_start")
    prices = c.fetchall()
    c.execute(f"SELECT * FROM sales WHERE product='{product}' ORDER BY date")
    sales = c.fetchall()
    i=0
    revenue =[]

    for j in range(len(sales)):
        if sales[j][1] > prices[i][0]:
            revenue.append(sales[j][2]*prices[i][2])
            if i != len(prices)-1:
                i+=1
            else:
                continue
        elif prices[i][0] == sales[j][1]:
            revenue.append(sales[j][2]*prices[i][2])
        else:
            revenue.append(sales[j][2]*prices[i-1][2])

    for i in range(len(revenue)):
        c.execute(f"""INSERT INTO revenue
                    VALUES ('{sales[i][0]}', '{sales[i][1]}', {sales[i][2]}, {revenue[i]})""")
        conn.commit()

In [ ]:
# population of revenue table
for i in range(len(product_names)):
    revenue_for_product(product_names[i][0])

In [461]:
c.execute("SELECT * FROM revenue WHERE product='B'")
B_data = c.fetchall()

In [462]:
# average
sum_1  = 0
for i in range(len(B_data)):
    sum_1 += B_data[i][3]
avg = sum_1/len(B_data)

# variance
sum_2 = 0
for i in range(len(B_data)):
    sum_2 += (B_data[i][3] - avg)**2
var = sum_2/len(B_data)

# standard deviation
sd = math.sqrt(var)

with open ('res2.txt', 'w') as w:
    w.write('%.2f' % sd)

In [463]:
print("Average value: %.2f" % avg)
print("Variance value: %.2f" % var)
print("Standard deviation value: %.2f" % sd)

Average value: 19030.89
Variance value: 1119651220.33
Standard deviation value: 33461.19


# Task 3

In [295]:
def lin_reg(product, date_start, date_end):
    
    c.execute(f"""SELECT date, amount FROM sales WHERE product='{product}'
                    AND date BETWEEN '{date_start}' AND '{date_end}' ORDER BY date""")
    data = c.fetchall()
    X, y = [], []
    for i in range(len(data)):
        X.append(i)
        y.append(data[i][1])
    X_df= pd.DataFrame(data=X, columns=['date'])
    y_df = pd.DataFrame(data=y, columns=['amount'])
    X_train = np.asanyarray(X_df[['date']])
    y_train = np.asanyarray(y_df[['amount']])
    LR = LinearRegression()
    LR.fit(X_train, y_train)
    
    return float('%.4f' % LR.coef_[0][0]), float('%.4f' % LR.intercept_[0])

In [297]:
coef = lin_reg('Q', '2019-01-01', '2019-03-31')
coef

(-0.1756, 29.3702)

In [303]:
with open ('res3.txt', 'w') as w:
    w.write(str(coef[0]) + ' ' + str(coef[1]))

In [451]:
conn.close()